In [79]:
from datetime import datetime
from math import pi
import calendar
import psycopg2, io, base64
from flask import Flask, render_template, request, url_for, redirect
import pandas as pd, matplotlib.pyplot as plt, plotly, json, numpy as np
import plotly.express as px
import matplotlib.ticker as mtick
from collections import defaultdict
import bokeh
from bokeh.palettes import Category10,Category20
from bokeh.plotting import figure
from bokeh.embed import components
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.transform import cumsum


app = Flask(__name__)

def bd(sql):
    conn = psycopg2.connect(host='localhost', database='IPA1', user='postgres', password='19122022')
    cur = conn.cursor()
    cur.execute(sql)
    dados = cur.fetchall()
    cur.close()
    conn.close()
    return dados


def gerar_plot(fig):
    img = io.BytesIO()
    plt.tight_layout()
    fig.savefig(img, format='png')
    img.seek(0)
    plot_url = base64.b64encode(img.getvalue()).decode()
    plt.close(fig)
    return plot_url

def carrega_menus():
    sql = '''
    SELECT m.id_menu, m.name_menu, s.id_submenu, s.name_submenu
    FROM menu m
    LEFT JOIN submenu s ON s.id_menu = m.id_menu
    ORDER BY m.ordem, s.id_submenu;
    '''
    dados = bd(sql)

    submenu_links = {
        'Crimes': 'dashboard_crimes',
        'Faccionado': 'dashboard_faccionado',
        'Crimes Por Cidade': 'dashboard_cidades',
        'Por Cidade': 'dashboard_custos',
        'Criminosos': 'dashboard_criminosos',
        'Anual': 'dashboard_mes_ano',
        'Por Crime': 'dashboard_custos_por_crime',
        'Por Ano': 'dashboard_custos_temporais',
        'Geral': 'dashboard_geral',
        'Crime': 'relatorio_crime'
    }

    menu_dict = {}

    for id_menu, name_menu, id_submenu, name_submenu in dados:
        if name_menu not in menu_dict:
            menu_dict[name_menu] = []
        
        if name_submenu:
            rota = submenu_links.get(name_submenu)
            menu_dict[name_menu].append((name_submenu, rota))
    
    return menu_dict

@app.context_processor
def inject_menus():
    menus = carrega_menus()
    home_link = ('Home', 'home')  # Nome e rota do link Home
    return {'menus': menus, 'home_link': home_link}

@app.route('/', methods=('GET', 'POST'))
def autentic():
    if request.method == 'POST':
        conta = request.form['login']
        senha = request.form['senha']
        sql="select * from administrador.usuario where (login='"+conta+"' and senha='"+senha+"')"
        #sql="select * from administrador.usuario where (conta='"+conta+"')"
        record=bd(sql)
        nr=len(record)
        if nr>=1:
            return redirect(url_for('home'))
        else:
            return redirect(url_for('indexnulo'))
    return render_template('autentic.html')

@app.route('/nulo')
def indexnulo():
    return render_template('indexnulo.html')



@app.route('/home')
def home():
    sql = '''
    SELECT m.id_menu, m.name_menu, s.id_submenu, s.name_submenu
    FROM menu m
    LEFT JOIN submenu s ON s.id_menu = m.id_menu
    ORDER BY m.ordem, s.id_submenu;
    '''
    dados = bd(sql)

    submenu_links = {
        'Crimes': 'dashboard_crimes',
        'Faccionado': 'dashboard_faccionado',
        'Crimes Por Cidade': 'dashboard_cidades',
        'Por Cidade': 'dashboard_custos',
        'Criminosos': 'dashboard_criminosos',
        'Anual': 'dashboard_mes_ano',
        'Por Crime': 'dashboard_custos_por_crime',
        'Por Ano': 'dashboard_custos_temporais',
        'Geral': 'dashboard_geral',
        'Crime': 'relatorio_crime'
    }

    menu_dict = {}

    for id_menu, name_menu, id_submenu, name_submenu in dados:
        if name_menu not in menu_dict:
            menu_dict[name_menu] = []
        
        if name_submenu:
            rota = submenu_links.get(name_submenu, '#')
            menu_dict[name_menu].append((name_submenu, rota))
    
    return render_template('homepage.html')


@app.route('/t1')
def index():
    sql='select * from menu'
    menus= bd(sql)
    return render_template('index.html', menus=menus)

@app.route('/t2')
def index1():
    sql='select * from submenu'
    menus= bd(sql)
    return render_template('index1.html', menus=menus)



@app.route('/dashboard/crimes')
def dashboard_crimes():
    sql = '''
        SELECT c.nome_crime, COUNT(co.id_ocorrencia) AS total
        FROM crime c
        JOIN crime_ocorrencia co ON c.id_crime = co.id_crime
        GROUP BY c.nome_crime;
    '''
    dados = bd(sql)
    df = pd.DataFrame(dados, columns=['Crime', 'Total'])

    fig,ax = plt.subplots(figsize=(10,6))
    ax.bar(df['Crime'],df['Total'],color='skyblue')
    ax.set_title('Ocorrências por Tipo de Crime')
    ax.set_xlabel('Tipo de Crime')
    ax.set_ylabel('Total de Ocorrências')
    plt.xticks(rotation=45)

    plot_url = gerar_plot(fig)

    return render_template("grafico_crimes.html", plot_url=plot_url, titulo="Crimes por Tipo")


@app.route('/dashboard/faccionado')
def dashboard_faccionado():
    sql = '''
        SELECT faccionado::text, COUNT(*) 
        FROM criminoso
        GROUP BY faccionado;
    '''
    dados = bd(sql)
    df = pd.DataFrame(dados, columns=['Faccionado', 'Total'])

    labels = ['Faccionado' if f == 'true' else 'Não Faccionado' for f in df['Faccionado']]
    fig, ax = plt.subplots()
    ax.pie(df['Total'], autopct='%1.1f%%', startangle=140)
    ax.set_title('Distribuição de Faccionados')
    ax.legend(labels,title='Categoria')

    plot_url = gerar_plot(fig)
    return render_template("grafico_faccionado.html", plot_url=plot_url,titulo='Criminosos Faccionados')


@app.route('/dashboard/cidades')
def dashboard_cidades():
    sql = '''
        SELECT cidade, COUNT(*) 
        FROM ocorrencia 
        GROUP BY cidade;
    '''
    dados = bd(sql)
    df = pd.DataFrame(dados, columns=['Cidade', 'Total'])

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(df['Cidade'], df['Total'], color='orange')
    ax.set_title('Crimes por Cidade')
    ax.set_xlabel('Cidade')
    ax.set_ylabel('Total de Ocorrências')
    plt.xticks(rotation=45)

    plot_url = gerar_plot(fig)
    return render_template("grafico_cidades.html", plot_url=plot_url, titulo='Crimes por Cidades')


@app.route('/dashboard/custos')
def dashboard_custos():
    sql = '''
        SELECT cidade, SUM(custo_operacional)
        FROM ocorrencia
        GROUP BY cidade;
    '''
    dados = bd(sql)
    df = pd.DataFrame(dados, columns=['Cidade', 'Custo Total'])

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(df['Cidade'], df['Custo Total'], color='green')
    ax.set_title('Custo Operacional por Cidade')
    ax.set_xlabel('Cidade')
    ax.set_ylabel('Custo Total')
    plt.xticks(rotation=45)

    plot_url = gerar_plot(fig)
    return render_template("grafico_custos.html", plot_url=plot_url, titulo='Custo gasto por Cidade')


@app.route('/dashboard/naturalidade')
def dashboard_criminosos():
    sql = '''
        SELECT naturalidade, COUNT(*)
        FROM criminoso
        GROUP BY naturalidade;
    '''
    dados = bd(sql)
    df = pd.DataFrame(dados, columns=['Naturalidade', 'Total'])

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(df['Naturalidade'], df['Total'], color='purple')
    ax.set_title('Criminosos por Naturalidade')
    ax.set_xlabel('Naturalidade')
    ax.set_ylabel('Total')
    plt.xticks(rotation=45)

    plot_url = gerar_plot(fig)
    return render_template("grafico_criminosos.html", plot_url=plot_url, titulo='Naturalidade Criminosos')

@app.route('/dashboard/mes_ano')
def dashboard_mes_ano():
    sql = '''
        SELECT 
            TO_CHAR(o.data_ocorrencia, 'YYYY-MM') AS ano_mes,
            COUNT(*) AS total
        FROM crime_ocorrencia co
        JOIN ocorrencia o ON co.id_ocorrencia = o.id_ocorrencia
        WHERE o.data_ocorrencia IS NOT NULL
        GROUP BY ano_mes
        ORDER BY ano_mes;
    '''
    dados = bd(sql)
    df = pd.DataFrame(dados, columns=['Ano_Mês', 'Total'])

    df['Mês_Ano'] = pd.to_datetime(df['Ano_Mês']).dt.strftime('%b/%Y')

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(df['Mês_Ano'], df['Total'], color='purple')
    ax.fill_between(df['Mês_Ano'], df['Total'], color='plum', alpha=0.3)


    ax.set_title('Crimes por Mês e Ano')
    ax.set_xlabel('Mês/Ano')
    ax.set_ylabel('Total de Ocorrências')
    plt.xticks(rotation=45)

    plot_url = gerar_plot(fig)
    return render_template("grafico_mes_ano.html", plot_url=plot_url, titulo='Crimes por Mês e Ano')


@app.route('/dashboard/custos_por_crime')
def dashboard_custos_por_crime():
    sql = '''
        SELECT c.nome_crime, SUM(o.custo_operacional) AS custo_total
        FROM crime_ocorrencia co JOIN crime c ON co.id_crime = c.id_crime
        JOIN ocorrencia o ON co.id_ocorrencia = o.id_ocorrencia
        GROUP BY c.nome_crime;

    '''
    dados = bd(sql)
    df = pd.DataFrame(dados, columns=['Crime', 'Custo Total'])

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.barh(df['Crime'], df['Custo Total'], color='red')
    ax.set_title('Custo Operacional por Tipo de Crime')
    ax.set_xlabel('Custo Total')
    ax.set_ylabel('Tipo de Crime')
    ax.invert_yaxis()

    plt.tight_layout()
    plot_url = gerar_plot(fig)
    return render_template("grafico_custos_por_crime.html", plot_url=plot_url, titulo='Custos por Crime')


@app.route('/dashboard/custos_temporais')
def dashboard_custos_temporais():
    sql = '''
        SELECT 
    EXTRACT(YEAR FROM o.data_ocorrencia)::int AS ano,
    TO_CHAR(o.data_ocorrencia, 'MM') AS mes,
    SUM(o.custo_operacional) AS total
    FROM 
    ocorrencia o
    WHERE 
    o.data_ocorrencia IS NOT NULL AND 
    EXTRACT(YEAR FROM o.data_ocorrencia) IN (2021, 2022)
    GROUP BY 
    ano, mes
    ORDER BY 
    ano, mes;
    '''
    dados = bd(sql)
    df = pd.DataFrame(dados, columns=['Ano', 'Mes', 'Total'])

    df_pivot = df.pivot(index='Mes', columns='Ano', values='Total').fillna(0)
    df_pivot = df_pivot.sort_index()

    meses = df_pivot.index
    largura = 0.35
    x = range(len(meses))

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar([i - largura/2 for i in x], df_pivot[2021], width=largura, label='2021', color='royalblue')
    ax.bar([i + largura/2 for i in x], df_pivot[2022], width=largura, label='2022', color='seagreen')

    ax.set_title('Comparativo de Custos Operacionais: 2021 vs 2022')
    ax.set_xlabel('Mês')
    ax.set_ylabel('Custo Total')
    ax.set_xticks(list(x))
    ax.set_xticklabels(meses)
    ax.legend()

    plt.tight_layout()
    plot_url = gerar_plot(fig)

    return render_template('dashboard_custos_temporais.html', plot_url=plot_url, titulo='Comparativo 2021 vs 2022')


@app.route('/relatorio/crime')
def relatorio_crime():
    sql = '''
        SELECT 
            c.nome_crime, 
            COUNT(co.id_ocorrencia) AS total_ocorrencias,
            AVG(o.tempo_prisao) AS media_prisao,
            SUM(o.custo_operacional) AS custo_total,
            AVG(o.custo_operacional) as media_custo
        FROM crime c
        JOIN crime_ocorrencia co ON c.id_crime = co.id_crime
        JOIN ocorrencia o ON co.id_ocorrencia = o.id_ocorrencia
        GROUP BY c.nome_crime;
    '''
    dados = bd(sql)
    colunas = ['Crime', 'Ocorrências', 'Tempo Médio de Prisão - anos', 'Custo Total - R$', 'Custo Médio - R$']
    return render_template("relatorio_crime.html", dados=dados, colunas=colunas, titulo="Relatório por Tipo de Crime")


@app.route('/bi')
def dashboard_geral():
    graficos = []

    def add_hover(fig, tooltips):
        hover = HoverTool(tooltips=tooltips)
        fig.add_tools(hover)
        return fig
    


    # 1) Ocorrências por Tipo de Crime
    sql1 = '''
        SELECT c.nome_crime, COUNT(co.id_ocorrencia)
        FROM crime c
        JOIN crime_ocorrencia co ON c.id_crime = co.id_crime
        GROUP BY c.nome_crime;
    '''
    df1 = pd.DataFrame(bd(sql1), columns=['Crime', 'Total'])
    source1 = ColumnDataSource(df1)

    p1 = figure(x_range=df1['Crime'], height=350, title="Ocorrências por Tipo de Crime", tools="")
    p1.vbar(x='Crime', top='Total', width=0.9, source=source1, fill_color='royalblue')

    hover1 = HoverTool(tooltips=[
        ("Crime", "@Crime"),
        ("Ocorrências", "@Total")
    ])
    p1.add_tools(hover1)


    p1.xgrid.grid_line_color = None
    p1.y_range.start = 0
    p1.xaxis.major_label_orientation = pi/4
    script1, div1 = components(p1)
    graficos.append(('Ocorrências por Tipo de Crime', script1, div1))

    # 2) Distribuição de Faccionados
    sql2 = '''
        SELECT faccionado::text, COUNT(*) 
        FROM criminoso
        GROUP BY faccionado;
    '''
    df2 = pd.DataFrame(bd(sql2), columns=['Faccionado', 'Total'])
    df2['angle'] = df2['Total'] / df2['Total'].sum() * 2 * pi
    df2['percent'] = df2['Total'] / df2['Total'].sum()
    cores_base = ['#718dbf', '#e84d60', '#ddb7b1', '#a6bddb']
    df2['color'] = [cores_base[i % len(cores_base)] for i in range(len(df2))]
    source2 = ColumnDataSource(df2)

    p2 = figure(height=350, title="Distribuição de Faccionados", toolbar_location=None,
                tools="", x_range=(-0.5, 1.0))
    p2.wedge(x=0, y=1, radius=0.4,
             start_angle=cumsum('angle', include_zero=True),
             end_angle=cumsum('angle'),
             line_color="white", fill_color='color', legend_field='Faccionado', source=source2)

    hover2 = HoverTool(tooltips=[
        ("Faccionado", "@Faccionado"),
        ("Total", "@Total"),
        ("Porcentagem", "@percent{0.00%}")
    ])
    p2.add_tools(hover2)

    p2.axis.visible = False
    p2.grid.grid_line_color = None
    script2, div2 = components(p2)
    graficos.append(('Distribuição de Faccionados', script2, div2))

    # 3) Crimes por Cidade
    sql3 = '''
        SELECT cidade, COUNT(*) 
        FROM ocorrencia 
        GROUP BY cidade;
    '''
    df3 = pd.DataFrame(bd(sql3), columns=['Cidade', 'Total'])
    source3 = ColumnDataSource(df3)

    p3 = figure(x_range=df3['Cidade'], height=350, title="Crimes por Cidade", tools="")
    p3.vbar(x='Cidade', top='Total', width=0.9, source=source3, fill_color='darkorange')

    hover3 = HoverTool(tooltips=[
        ("Cidade", "@Cidade"),
        ("Crimes", "@Total")
    ])
    p3.add_tools(hover3)

    p3.xgrid.grid_line_color = None
    p3.y_range.start = 0
    p3.xaxis.major_label_orientation = pi/4
    script3, div3 = components(p3)
    graficos.append(('Crimes por Cidade', script3, div3))

    # 4) Custo Operacional por Cidade
    sql4 = '''
        SELECT cidade, SUM(custo_operacional)
        FROM ocorrencia
        GROUP BY cidade;
    '''
    df4 = pd.DataFrame(bd(sql4), columns=['Cidade', 'Custo Total'])
    source4 = ColumnDataSource(df4)

    p4 = figure(x_range=df4['Cidade'], height=350, title="Custo Operacional por Cidade", tools="")
    p4.vbar(x='Cidade', top='Custo Total', width=0.9, source=source4, fill_color='seagreen')

    hover4 = HoverTool(tooltips=[
        ("Cidade", "@Cidade"),
        ("Custo Total", "@{Custo Total}{R$ 0,0.00}")
    ])
    p4.add_tools(hover4)

    p4.xgrid.grid_line_color = None
    p4.y_range.start = 0
    p4.xaxis.major_label_orientation = pi/4
    script4, div4 = components(p4)
    graficos.append(('Custo Operacional por Cidade', script4, div4))

    # 5) Crimes por Mês e Ano
    sql5 = '''
        SELECT TO_CHAR(o.data_ocorrencia, 'YYYY-MM') AS ano_mes, COUNT(*)
        FROM crime_ocorrencia co
        JOIN ocorrencia o ON co.id_ocorrencia = o.id_ocorrencia
        WHERE o.data_ocorrencia IS NOT NULL
        GROUP BY ano_mes ORDER BY ano_mes;
    '''
    df5 = pd.DataFrame(bd(sql5), columns=['Ano_Mês', 'Total'])
    df5['Mês_Ano'] = pd.to_datetime(df5['Ano_Mês']).dt.strftime('%b/%Y')
    source5 = ColumnDataSource(df5)

    p5 = figure(x_range=df5['Mês_Ano'], height=350, title="Crimes por Mês e Ano", tools="")
    p5.line(x='Mês_Ano', y='Total', source=source5, line_width=2, color='purple')
    p5.circle(x='Mês_Ano', y='Total', source=source5, size=8, color='purple')

    hover5 = HoverTool(tooltips=[
        ("Mês/Ano", "@Mês_Ano"),
        ("Total", "@Total")
    ])
    p5.add_tools(hover5)

    p5.xaxis.major_label_orientation = pi/4
    script5, div5 = components(p5)
    graficos.append(('Crimes por Mês e Ano', script5, div5))


    df6 = pd.DataFrame(bd('''
        SELECT c.categoria, COUNT(co.id_ocorrencia) AS total
        FROM crime c
        JOIN crime_ocorrencia co ON c.id_crime = co.id_crime
        GROUP BY c.categoria;
    '''), columns=["Categoria", "Total"])
    src6 = ColumnDataSource(df6)
    p6 = figure(y_range=df6["Categoria"], width=600, height=350, title="Crimes por Categoria", tools="")
    p6.hbar(y="Categoria", right="Total", height=0.8, source=src6, fill_color="cadetblue")
    p6.x_range.start = 0
    add_hover(p6, [("Categoria", "@Categoria"), ("Total", "@Total")])
    p6.xaxis.major_label_orientation = pi/4
    script6, div6 = components(p6)
    graficos.append(("Crimes por Categoria", script6, div6))

    # 2. Tempo médio de prisão por tipo de crime
    df7 = pd.DataFrame(bd('''
        SELECT c.nome_crime, AVG(o.tempo_prisao) AS media_tempo
        FROM crime c
        JOIN crime_ocorrencia co ON c.id_crime = co.id_crime
        JOIN ocorrencia o ON co.id_ocorrencia = o.id_ocorrencia
        GROUP BY c.nome_crime;
    '''), columns=["Crime", "Tempo_Médio"])
    df7.columns = ["Crime", "TempoMedio"]
    src7 = ColumnDataSource(df7)
    print(df7.head())
    print(df7.dtypes)
    p7 = figure(x_range=df7["Crime"], height=350, title="Tempo Médio de Prisão por Tipo de Crime", tools="")
    p7.vbar(x="Crime", top="TempoMedio", width=0.9, source=src7, fill_color="indianred")
    p7.add_tools(HoverTool(tooltips=[("Crime", "@Crime"),("Média (h)", "@TempoMedio{0.00}")]))
    p7.xaxis.major_label_orientation = pi/4
    script7, div7 = components(p7)
    graficos.append(("Tempo Médio de Prisão por Tipo de Crime", script7, div7))

    # 3. Criminosos por Naturalidade
    df8 = pd.DataFrame(bd('''
        SELECT naturalidade, COUNT(*) AS total
        FROM criminoso
        GROUP BY naturalidade;
    '''), columns=["Naturalidade", "Total"])
    src8 = ColumnDataSource(df8)
    p8 = figure(x_range=df8["Naturalidade"], height=350, title="Criminosos por Naturalidade", tools="")
    p8.vbar(x="Naturalidade", top="Total", width=0.9, source=src8, fill_color="darkorange")
    add_hover(p8, [("Naturalidade", "@Naturalidade"), ("Total", "@Total")])
    p8.xaxis.major_label_orientation = pi/4
    script8, div8 = components(p8)
    graficos.append(("Criminosos por Naturalidade", script8, div8))

    # 4. Faccionados por Cidade da Ocorrência
    df9 = pd.DataFrame(bd('''
        SELECT o.cidade, COUNT(*) AS total
        FROM criminoso c
        JOIN criminoso_ocorrencia co ON c.cpf = co.cpf
        JOIN ocorrencia o ON co.id_ocorrencia = o.id_ocorrencia
        WHERE c.faccionado = TRUE
        GROUP BY o.cidade;
    '''), columns=["Cidade", "Total"])
    src9 = ColumnDataSource(df9)
    p9 = figure(x_range=df9["Cidade"], height=350, title="Faccionados por Cidade da Ocorrência", tools="")
    p9.vbar(x="Cidade", top="Total", width=0.9, source=src9, fill_color="seagreen")
    add_hover(p9, [("Cidade", "@Cidade"), ("Faccionados", "@Total")])
    p9.xaxis.major_label_orientation = pi/4
    script9, div9 = components(p9)
    graficos.append(("Faccionados por Cidade da Ocorrência", script9, div9))

    # 5. Custo Operacional por Tipo de Crime
    

    df10 = pd.DataFrame(bd('''
    SELECT c.nome_crime, SUM(o.custo_operacional) AS total_custo
    FROM crime c
    JOIN crime_ocorrencia co ON c.id_crime = co.id_crime
    JOIN ocorrencia o ON co.id_ocorrencia = o.id_ocorrencia
    GROUP BY c.nome_crime;
    '''), columns=["Crime", "Custo"])

    df10['angle'] = df10['Custo'] / df10['Custo'].sum() * 2 * pi

    n = len(df10)
    paleta = Category20[20] if n > 20 else Category20[n]
    df10['color'] = paleta[:n]

    src10 = ColumnDataSource(df10)

    p10 = figure(height=350, title="Custo Operacional por Tipo de Crime",
                tools="", x_range=(-0.5, 1.0))

    p10.wedge(x=0, y=1, radius=0.4,
            start_angle=cumsum('angle', include_zero=True),
            end_angle=cumsum('angle'),
            line_color="white", fill_color='color', legend_field='Crime', source=src10)

    # Buraco no meio do gráfico (anello branco)
    p10.annulus(x=0, y=1, inner_radius=0.2, outer_radius=0.35, color='white')

    hover = HoverTool(tooltips=[
        ("Crime", "@Crime"),
        ("Custo Total", "@Custo{R$ 0,0.00}")
    ])
    p10.add_tools(hover)

    p10.axis.visible = False
    p10.grid.grid_line_color = None
    p10.legend.location = "top_right"
    p10.legend.label_text_font_size = "8pt"

    script10, div10 = components(p10)
    graficos.append(("Custo Operacional por Tipo de Crime", script10, div10))





    dados_idade = bd('''
        SELECT data_nasc FROM criminoso WHERE data_nasc IS NOT NULL
    ''')
    df_idade = pd.DataFrame(dados_idade, columns=['data_nasc'])

    hoje = datetime.today()
    df_idade['idade'] = df_idade['data_nasc'].apply(lambda d: (hoje.date() - d).days // 365)

    bins = list(range(0, 101, 10))  # Faixas de 10 em 10 anos
    hist, edges = np.histogram(df_idade['idade'], bins=bins)

    source_idade = ColumnDataSource(data=dict(
        top=hist,
        left=edges[:-1],
        right=edges[1:]
    ))

    p_idade = figure(title="Distribuição de Idade dos Criminosos",
                    x_axis_label='Idade (anos)',
                    y_axis_label='Número de Criminosos',
                    height=350,
                    tools="hover")

    p_idade.quad(top='top', bottom=0, left='left', right='right', source=source_idade,
                fill_color='navy', line_color='white', alpha=0.7)

    p_idade.xaxis.ticker = bins
    p_idade.xaxis.major_label_orientation = pi/4

    hover = p_idade.select_one(HoverTool)
    hover.tooltips = [("Faixa", "@left - @right"), ("Quantidade", "@top")]

    script_idade, div_idade = components(p_idade)
    graficos.append(("Distribuição de Idade dos Criminosos", script_idade, div_idade))


    return render_template("index2.html", graficos=graficos)



if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
